In [275]:
import pandas as pd
import numpy as np
import os
import scipy.cluster.hierarchy as sch
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from sklearn.metrics import *
from collections import Counter

In [282]:
class HierarchicalClustering:
    def __init__(self, state: str):
        self.state = state
        self.ACs = []
        for ac_name in os.listdir('./ACs'):
            if state in ac_name:
                ac_i = pd.read_pickle('./ACs/'+ac_name)
                self.ACs.append(ac_i)
        self.n_rows = len(self.ACs[0])

    def get_X_Y(self, row: int) -> (pd.DataFrame, pd.DataFrame):
        data = []
        for AC in self.ACs:
            AC = pd.DataFrame(AC)
            data.append(AC.iloc[row,:].tolist())
        data = pd.DataFrame(data).reset_index(drop=True)
        X = data.iloc[:,1:]
        Y = data.iloc[:,0]
        
        return X, Y    
    
    def cluster(self, X: pd.DataFrame, Y: pd.Series, method='ward') -> (list, list):
        dendrogram = sch.dendrogram(sch.linkage(X, method=method), no_plot=True)
        
        leaves = dendrogram['leaves']
        leaves_color_list = dendrogram['leaves_color_list']
        color_counts = Counter(dendrogram['leaves_color_list'])
        
        n_leaves = len(leaves_color_list)
        
        anomaly_score = [0] * n_leaves
        for i, color in zip(leaves, leaves_color_list):
            anomaly_score[i] = 1 - (color_counts[color]/n_leaves)
            
        return anomaly_score, Y.astype(int).tolist()
    
    def cluster_all(self, do_score=False, method='ward') -> (list, list):
        
        anomaly_score, y = [], []
        
        for row in range(self.n_rows):
            X, Y = hc.get_X_Y(row)
            if sum(Y) > len(Y)//2: continue
                
            anomaly_score_row, y_row  = hc.cluster(X,Y,method)
            
            anomaly_score += anomaly_score_row
            y += y_row
                    
        return anomaly_score, y
    
    def score(self, anomaly_score, y, threashold):
        
        yhat = [ 0 if y_i < threashold else 1 for y_i in anomaly_score]
            
        # accuracy: (tp + tn) / (p + n)
        accuracy = accuracy_score(y, yhat)
        print('Accuracy: %f' % accuracy)
        # precision tp / (tp + fp)
        precision = precision_score(y, yhat)
        print('Precision: %f' % precision)
        # recall: tp / (tp + fn)
        recall = recall_score(y, yhat)
        print('Recall: %f' % recall)
        # f1: 2 tp / (2 tp + fp + fn)
        f1 = f1_score(y, yhat)
        print('F1 score: %f' % f1)
        # AUC
        fpr, tpr, thresholds = roc_curve(y, anomaly_score)
        AUC = auc(fpr, tpr)
        print('AUC score: %f' % AUC)
        return yhat, y, anomaly_score
        

In [283]:
hc = HierarchicalClustering('harmonic_low_18_cold_fullyOpen')
anomaly_score, y = hc.cluster_all()

In [284]:
for threashold in [6/11, 7/11, 8/11, , 0.85, 0.9]:
    print('Threashold : ' + str(threashold))
    hc.score(anomaly_score, y, threashold)
    print('--------------------')

Threashold : 0.5
Accuracy: 0.855636
Precision: 0.406728
Recall: 0.967273
F1 score: 0.572659
AUC score: 0.949905
--------------------
Threashold : 0.6
Accuracy: 0.888000
Precision: 0.468809
Recall: 0.901818
F1 score: 0.616915
AUC score: 0.949905
--------------------
Threashold : 0.7
Accuracy: 0.915636
Precision: 0.557333
Recall: 0.760000
F1 score: 0.643077
AUC score: 0.949905
--------------------
Threashold : 0.8
Accuracy: 0.917091
Precision: 0.605856
Recall: 0.489091
F1 score: 0.541247
AUC score: 0.949905
--------------------
Threashold : 0.85
Accuracy: 0.909455
Precision: 0.664557
Recall: 0.190909
F1 score: 0.296610
AUC score: 0.949905
--------------------
Threashold : 0.9
Accuracy: 0.909455
Precision: 0.664557
Recall: 0.190909
F1 score: 0.296610
AUC score: 0.949905
--------------------


In [288]:
anomaly_score

[0.09090909090909094,
 0.09090909090909094,
 0.09090909090909094,
 0.09090909090909094,
 0.09090909090909094,
 0.09090909090909094,
 0.09090909090909094,
 0.9090909090909091,
 0.09090909090909094,
 0.09090909090909094,
 0.09090909090909094,
 0.6363636363636364,
 0.6363636363636364,
 0.36363636363636365,
 0.6363636363636364,
 0.36363636363636365,
 0.36363636363636365,
 0.36363636363636365,
 0.36363636363636365,
 0.6363636363636364,
 0.36363636363636365,
 0.36363636363636365,
 0.09090909090909094,
 0.09090909090909094,
 0.09090909090909094,
 0.09090909090909094,
 0.09090909090909094,
 0.09090909090909094,
 0.09090909090909094,
 0.09090909090909094,
 0.09090909090909094,
 0.09090909090909094,
 0.9090909090909091,
 0.36363636363636365,
 0.36363636363636365,
 0.6363636363636364,
 0.6363636363636364,
 0.36363636363636365,
 0.6363636363636364,
 0.36363636363636365,
 0.36363636363636365,
 0.36363636363636365,
 0.36363636363636365,
 0.6363636363636364,
 0.09090909090909094,
 0.9090909090909091,

In [291]:
yhat = [ 1 if y_i >= 8/11 else 0 for y_i in anomaly_score]

In [292]:
# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(y, yhat)
print('Accuracy: %f' % accuracy)
# precision tp / (tp + fp)
precision = precision_score(y, yhat)
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(y, yhat)
print('Recall: %f' % recall)
# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(y, yhat)
print('F1 score: %f' % f1)

Accuracy: 0.915636
Precision: 0.557333
Recall: 0.760000
F1 score: 0.643077


In [293]:
8/11


0.7272727272727273